# Desafio - Extraindo informações de receitas para uma lista de compras

### A partir da receita, você deve extrair as seguintes informações:

* ``Utensílios de Cozinha:`` Liste todos os utensílios que são mencionados na receita (por exemplo, liquidificador, tigela, batedeira).
* ``Ingredientes:`` Liste todos os ingredientes necessários para preparar o bolo (por exemplo, cenoura, ovos, óleo, açúcar, farinha de trigo, fermento, manteiga, chocolate em pó, leite).
* ``Salvar em CSV:`` Após extrair as informações, salve os dados de ingredientes em um arquivo CSV e os utensílios em outro.

In [14]:
from langchain_core.utils.function_calling import convert_to_openai_function
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate as cpt
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())

receita = '''Receita de Bolo de Cenoura
Massa

Em um liquidificador, adicione a cenoura, os ovos e o óleo, depois misture.
Acrescente o açúcar e bata novamente por 5 minutos.
Em uma tigela ou na batedeira, adicione a farinha de trigo e depois misture novamente.
Acrescente o fermento e misture lentamente com uma colher.
Asse em um forno preaquecido a 180° C por aproximadamente 40 minutos.
Cobertura
Despeje em uma tigela a manteiga, o chocolate em pó, o açúcar e o leite, depois misture.
Leve a mistura ao fogo e continue misturando até obter uma consistência cremosa, depois despeje a calda por cima do bolo.
'''

class UtensiliosIngredientes(BaseModel):
    '''Lista de utensílios e ingredientes necessários para fazer uma receita'''
    utensilios: str = Field(description='Lista de utensílios de cozinha necessários para fazer a receita', examples=['Liquidificador', 'Tigela', 'Batedeira'])
    ingredientes: str = Field(description='Lista de ingredientes necessários para fazer a receita', examples=['Farinha', 'Ovos', 'Leite'])

tool_compras = convert_to_openai_function(UtensiliosIngredientes)

prompt = cpt.from_messages([
    ('system', 'Extraia dessa receita todos os utensílios de cozinha e ingredientes necessários'),
    ('user', '{input}')
])

chat = ChatOpenAI(model='gpt-4o-mini')
chat = chat.bind(functions=[tool_compras], function_call={'name': 'UtensiliosIngredientes'})

chain = prompt | chat | JsonOutputFunctionsParser()

chain.invoke({'input': receita})

{'utensilios': 'Liquidificador, Tigela, Batedeira, Colher, Forma para assar, Panela',
 'ingredientes': 'Cenoura, Ovos, Óleo, Açúcar, Farinha de trigo, Fermento, Manteiga, Chocolate em pó, Leite'}